## Model Training

### importing necessary libraries

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# modeling libraries
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

In [12]:
X = pd.read_csv('./data/feature_matrix.csv')
X.head()

,country_freq,year,city_freq,main_category_freq,funding_rounds,funding_filled,first_funding_year,last_funding_year,funding_duration_days,avg_funding_per_round
0,1736,2015,288,226,1,10000000,2015,2015,0,9.999900e+06
1,40920,2014,5,228,2,700000,2014,2014,227,3.499983e+05
2,4009,2014,7927,1501,1,3406878,2014,2014,0,3.406844e+06
3,1714,2007,573,2181,1,2000000,2008,2008,0,1.999980e+06
4,40920,2010,32,4010,1,10070591,2014,2014,0,1.007049e+07


In [13]:
y = pd.read_csv('./data/target_variable.csv')
y.head()

,success
0,False
1,False
2,False
3,False
4,False


In [14]:
y = y.values.ravel() 

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [17]:
scaler = StandardScaler()
X_train_res = scaler.fit_transform(X_train_res)
X_test = scaler.transform(X_test)


In [21]:
log_reg = LogisticRegression(class_weight='balanced', random_state=42)
log_reg.fit(X_train_res, y_train_res)
y_pred_log_reg = log_reg.predict(X_test)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_log_reg))
print("accuracy:", log_reg.score(X_test, y_test))
print(confusion_matrix(y_test, y_pred_log_reg))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

       False       0.95      0.78      0.86     11657
        True       0.25      0.66      0.36      1315

    accuracy                           0.76     12972
   macro avg       0.60      0.72      0.61     12972
weighted avg       0.88      0.76      0.81     12972

accuracy: 0.7642614862781375
[[9049 2608]
 [ 450  865]]
